In [72]:
import math
import heapq

text = open("input.txt").read().split("\n")
text = [list(x) for x in text]
grid = {}
distances = []
visited = {}
parent = {}
actual_distances = {}
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

start = (0, 0)
end = (len(text) - 1, len(text[0]) - 1)


In [73]:
# Setup initial data structures

for i, row in enumerate(text):
    for j, col in enumerate(row):
        grid[(i, j)] = int(col)
        distances.append((math.inf, (i, j), (-1, -1)))
        for direction in directions:
            actual_distances[((i, j), direction)] = math.inf
            visited[((i, j), direction)] = False
            parent[((i, j), direction)] = ()


actual_distances[((0, 0), (-1, -1))] = 0
visited[((0, 0), (-1, -1))] = False
distances[0] = (0, (0, 0), (-1, -1))
heapq.heapify(distances)

In [74]:
def tsum(one, two, factor = 1):
    return (one[0] + factor * two[0], one[1] + factor * two[1])

In [75]:
while distances:
    distance, min_node, odirection = heapq.heappop(distances)
    if visited[(min_node, odirection)]:
        continue
    visited[(min_node, odirection)] = True
    if min_node == end:
        end = (min_node, odirection)
        break

    # Move in increments of a direction based on the problem
    for direction in directions:
        for i in range(1, 4):
            extended_direction = (i * direction[0], i * direction[1])
            check_point = tsum(min_node, extended_direction)
            # Out of bounds checking
            if check_point[0] < 0 or check_point[0] >= len(text):
                continue
            if check_point[1] < 0 or check_point[1] >= len(text[0]):
                continue
            # Get the exact points travelled and force a turn based on origin direction
            if odirection[0] == 0 and direction[0] == 0:
                continue
            else:
                if direction[0] != 0:
                    travelled_x = [grid[(min_node[0] + x, min_node[1])] for x in range(direction[0], extended_direction[0] + direction[0], direction[0])]
                else:
                    travelled_x = []
            if odirection[1] == 0 and direction[1] == 0:
                continue
            else:
                if direction[1] != 0:
                    travelled_y = [grid[(min_node[0], min_node[1] + x)] for x in range(direction[1], extended_direction[1] + direction[1], direction[1])]
                else:
                    travelled_y = []
            travelled = travelled_x if len(travelled_x) else travelled_y
            check_distance = sum(travelled)
            current_distance = actual_distances[(check_point, direction)]
            if distance + check_distance < current_distance:
                actual_distances[(check_point, direction)] = distance + check_distance
                heuristic = abs(end[0] - check_point[0]) + abs(end[1] - check_point[1])
                # print(end, check_point, heuristic, actual_distances[(check_point, direction)])
                heapq.heappush(distances, ((actual_distances[(check_point, direction)]), check_point, direction))
                parent[(check_point, direction)] = (min_node, odirection)

In [76]:
point = end
result = 0
intermediate_sum = 0
path = []
path.append(end[0])
while True:
    parent_node = parent[point]
    point = parent_node
    path.append(parent_node[0])
    if point[0] == start:
        break
    result += grid[point[0]]

path = path[::-1]
full_path = set()
result = 0
for i, point in enumerate(path[:-1]):
    next_point = path[i + 1]
    y_travel, x_travel = tsum(next_point, point, -1)
    x_direction = 1 if x_travel > 0 else -1
    y_direction = 1 if y_travel > 0 else -1
    full_path.add(point)
    for j in range(1, abs(x_travel)):
        intermediate_point = (point[0], point[1] + x_direction * j)
        full_path.add(intermediate_point)
    for k in range(1, abs(y_travel)):
        intermediate_point = (point[0] + y_direction * k, point[1])
        full_path.add(intermediate_point)
    full_path.add(next_point)

print(sum([grid[x] for x in list(full_path)]) - grid[start])

102
